In [16]:
import numpy as np
import pandas as pd
import requests
import zipfile
import altair as alt
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

#### Title 

#### Summary 

#### Introduction

- provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report

- clearly state the question you tried to answer with your project

- identify and describe the dataset that was used to answer the question

#### Methods and Results

1. describe in written english the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.

2. your report should include code which:
        - loads data from the original source on the web

        - wrangles and cleans the data from it’s original (downloaded) format to the format necessary for the planned classification or clustering analysis

        - performs a summary of the data set that is relevant for exploratory data analysis related to the planned classification analysis

        - creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned classification analysis

        - performs classification or regression analysis

        - creates a visualization of the result of the analysis

note: all tables and figure should have a figure/table number and a legend

In [17]:
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"

column_names = [
    "Sex", "Length", "Diameter", "Height",
    "Whole_weight", "Shucked_weight",
    "Viscera_weight", "Shell_weight", "Rings"
]
abalone = pd.read_csv(url, header=None, names=column_names)

abalone.to_csv("data/abalone.csv", index=False)

In [18]:
abalone

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


#### Discussion:
summarize what you found
discuss whether this is what you expected to find?
discuss what impact could such findings have?
discuss what future questions could this lead to?

#### References: 
at least 4 citations relevant to the project (format is your choose, just be consistent across the references).